<a href="https://colab.research.google.com/github/Anurag38/NLP-Practicals/blob/main/Practical%204%20-%20word2vec/word2vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from gensim.models import KeyedVectors
from gensim.models.wrappers import FastText
from gensim.models import FastText



In [2]:
import itertools
from gensim.models.word2vec import Text8Corpus
#from glove import Corpus, Glove
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
# from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
#from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
from tqdm import tqdm
tqdm.pandas(desc="progress-bar")
import gensim
from gensim.models.word2vec import Word2Vec
from gensim.models.doc2vec import TaggedDocument
import multiprocessing
from sklearn import utils
%matplotlib inline

In [3]:
# provide sql-like data manipulation tools. very handy.
import pandas as pd 
pd.options.mode.chained_assignment = None

# high dimensional vector computing library.
import numpy as np 
from copy import deepcopy
from string import punctuation
from random import shuffle

import gensim

# the word2vec model gensim class
from gensim.models.word2vec import Word2Vec 

# we'll talk about this down below
LabeledSentence = gensim.models.doc2vec.LabeledSentence 

from tqdm import tqdm
tqdm.pandas(desc="progress-bar")

# a tweet tokenizer from nltk.
from nltk.tokenize import TweetTokenizer 
tokenizer = TweetTokenizer()

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

In [4]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [5]:
train= pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/AMI/dataset/taska.txt',header=None,sep='\t',names=['id','text','class'])
#valid_df= pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/AMI/dataset/agr_en_dev.csv',header=None,names=['id','text','class'])

#test_df = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/AMI/dataset/agr_en_fb_gold.csv',header=None,names=['id','text','class'])


In [6]:
train.info()
train.head()
train['class'].value_counts()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13240 entries, 0 to 13239
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      13240 non-null  int64 
 1   text    13240 non-null  object
 2   class   13240 non-null  object
dtypes: int64(1), object(2)
memory usage: 310.4+ KB


NOT    8840
OFF    4400
Name: class, dtype: int64

In [7]:
import re
def clean_text(comment):
    comment = comment.strip().strip('"')
    comment = comment.replace('_', ' ')
    comment = comment.replace("\\\\", "\\")
    comment = comment.replace('\\n', ' ')
    comment = comment.replace('\\n', ' ')
    comment = comment.lower()
    comment = re.sub(r'@[A-Za-z0-9]+','',comment)
    
    comment = re.sub('https?://[A-Za-z0-9./]+','',comment)
    commment = re.sub("[^a-zA-Z]", " ", comment)
    comment = re.sub(r'[^\w\s]','',comment)

    
    return comment

train['text'] = train['text'].fillna("something").values
train['ctext']=train['text'].apply(clean_text)
#train_df['ctext']=train_df['text']
#valid_df['ctext']=valid_df['text'].apply(clean_text)
#test_df['ctext']=test_df['text'].apply(clean_text)
train.head()


,id,text,class,ctext
0,86426,@USER She should ask a few native Americans wh...,OFF,she should ask a few native americans what th...
1,90194,@USER @USER Go home you’re drunk!!! @USER #MAG...,OFF,go home youre drunk maga trump2020 url
2,16820,Amazon is investigating Chinese employees who ...,NOT,amazon is investigating chinese employees who ...
3,62688,"@USER Someone should'veTaken"" this piece of sh...",OFF,someone shouldvetaken this piece of shit to a...
4,43605,@USER @USER Obama wanted liberals &amp; illega...,NOT,obama wanted liberals amp illegals to move i...


In [8]:
#tr=train_df[train_df['text']=='something']
#tr.head()

In [9]:
#train_df['text'] = train_df['text'].fillna("something").values
train['class'] = train['class'].map({'OFF': 0, 'NOT': 1})


In [10]:
train_df, test_df = train_test_split(train,test_size = 0.1)


In [11]:
import nltk
nltk.download('punkt')
train_df['tokenized_sents'] = train_df.apply(lambda row: nltk.word_tokenize(row['ctext']), axis=1)
test_df['tokenized_sents'] = test_df.apply(lambda row: nltk.word_tokenize(row['ctext']), axis=1)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [12]:
train_df.head()

,id,text,class,ctext,tokenized_sents
7125,68984,@USER But you are? Give me a break!,1,but you are give me a break,"[but, you, are, give, me, a, break]"
12144,87051,@USER @USER @USER @USER Don't feel sorry for h...,0,dont feel sorry for her she is so greedy s...,"[dont, feel, sorry, for, her, she, is, so, gre..."
4283,62703,@USER Arrest everyone of them! We should be do...,1,arrest everyone of them we should be doing th...,"[arrest, everyone, of, them, we, should, be, d..."
12083,42482,10 restaurant diners fatally shot in Corpus Ch...,0,10 restaurant diners fatally shot in corpus ch...,"[10, restaurant, diners, fatally, shot, in, co..."
3287,86117,@USER You are the man Gregg!,1,you are the man gregg,"[you, are, the, man, gregg]"


In [13]:
x_train=train_df['tokenized_sents']
x_test=test_df['tokenized_sents']

y_train=train_df['class']
y_test=test_df['class']


In [14]:
def labelizeTweets(tweets, label_type):
    labelized = []
    for i,v in tqdm(enumerate(tweets)):
        label = '%s_%s'%(label_type,i)
        labelized.append(LabeledSentence(v, [label]))
    return labelized

x_train = labelizeTweets(x_train, 'TRAIN')

x_test = labelizeTweets(x_test, 'TEST')


0it [00:00, ?it/s]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `LabeledSentence` (Class will be removed in 4.0.0, use TaggedDocument instead).
  """
11916it [00:00, 243843.22it/s]
1324it [00:00, 7077.88it/s]


In [15]:
#tweet_w2v = FastText(size=400, min_count=1,sg=1)
tweet_w2v = Word2Vec(size=300, min_count=1,sg=1) #SKIPGRAM
tweet_w2v.build_vocab([x.words for x in tqdm(x_train)])
tweet_w2v.train([x.words for x in tqdm(x_train)],total_examples=len(x_train),epochs=10)


100%|██████████| 11916/11916 [00:00<00:00, 1540242.43it/s]


(1818227, 2334250)

In [16]:
tweet_w2v['hello']

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


array([-0.0678657 ,  0.14923333,  0.05878469,  0.05446722, -0.10443438,
       -0.13105269, -0.13153464,  0.0372822 , -0.01119888, -0.00892492,
       -0.07212088, -0.02818428, -0.04634731, -0.05089912, -0.07144504,
        0.01041827, -0.09932423,  0.08100637, -0.32556817,  0.01157103,
        0.0879048 ,  0.00543198, -0.08159877, -0.01675915, -0.06183286,
       -0.14255397, -0.07467941,  0.12077925, -0.05880748, -0.08940005,
       -0.08674736, -0.17640902,  0.1004069 ,  0.06550197, -0.15900809,
       -0.02836237, -0.00280109, -0.00492049, -0.10798977, -0.14996919,
        0.14022781,  0.0088025 ,  0.18589672,  0.09244545,  0.00438946,
        0.00231881,  0.06667206,  0.07980375,  0.12325242, -0.05567688,
       -0.02295933,  0.07490967,  0.02445438, -0.10410339,  0.0064341 ,
        0.2402889 , -0.10417822,  0.1718549 , -0.2294985 ,  0.00942066,
        0.06131065, -0.03393849, -0.13653105,  0.07537535,  0.04390161,
       -0.06119368, -0.1598253 , -0.0900723 ,  0.10706846,  0.03

In [17]:
tweet_w2v['anurag']

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


KeyError: ignored

In [ ]:
#tweet_w2v.save('off-sg.vec')
#tweet_w2v.save('fb-en-w2vec-cbow.bin')
#tweet_w2v.wv.save_word2vec_format('fb-hi-w2vec-cbow.txt')